# 06 - Registers: Fast CPU Storage

## Introduction

**Registers** are small, fast storage locations inside the CPU. They're built from D flip-flops and are used to:

- Hold operands for ALU operations
- Store intermediate results
- Keep track of memory addresses

In this notebook, we'll build:

1. **8-bit Register** - Holds a single byte
2. **Register File** - A collection of registers with addressing

## Learning Objectives

1. Understand how registers are built from flip-flops
2. Learn about register enable signals
3. Build a register file with read/write ports

In [ ]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))
sys.path.insert(0, str(project_root))

from computer.sequential import DFlipFlop
from computer import int_to_bits, bits_to_int
from typing import List

print("Setup complete!")

## Register Architecture

An 8-bit register is simply 8 D flip-flops sharing the same clock:

```mermaid
flowchart TB
    subgraph Data Input
        D0((D0)) --> DFF0[DFF]
        D1((D1)) --> DFF1[DFF]
        D2((D2)) --> DFF2[DFF]
        D3((...)) --> DFF3[...]
        D7((D7)) --> DFF7[DFF]
    end
    CLK((CLK)) --> DFF0
    CLK --> DFF1
    CLK --> DFF2
    CLK --> DFF3
    CLK --> DFF7
    DFF0 --> Q0((Q0))
    DFF1 --> Q1((Q1))
    DFF2 --> Q2((Q2))
    DFF3 --> Q3((...))
    DFF7 --> Q7((Q7))
```

The **enable** signal controls whether the register accepts new data:
- `enable=1`: Register captures input data on clock edge
- `enable=0`: Register holds current value

## Exercise 1: 8-bit Register

In [ ]:
class Register8:
    """8-bit register built from D flip-flops."""

    def __init__(self):
        self.bits = [DFlipFlop() for _ in range(8)]

    def clock(self, data: List[int], enable: int, clk: int) -> List[int]:
        """Update register on clock edge when enabled.

        Args:
            data: 8-bit input data (LSB at index 0)
            enable: 1 to allow write, 0 to hold
            clk: Clock signal

        Returns:
            Current register value
        """
        # YOUR CODE HERE
        # Only update flip-flops when enable=1
        pass

    def read(self) -> List[int]:
        """Read current register value."""
        # YOUR CODE HERE
        pass


# Test
reg = Register8()
print(f"Initial value: {bits_to_int(reg.read())}")

# Write 42 to register
data = int_to_bits(42, 8)
reg.clock(data, 1, 0)  # CLK low
reg.clock(data, 1, 1)  # Rising edge
print(f"After writing 42: {bits_to_int(reg.read())}")

# Try to write 100 with enable=0
data = int_to_bits(100, 8)
reg.clock(data, 0, 0)
reg.clock(data, 0, 1)
print(f"After trying to write 100 (disabled): {bits_to_int(reg.read())}")

## Register File Architecture

A **register file** is a collection of registers that can be accessed by address. Our CPU will have 8 registers (R0-R7), requiring a 3-bit address.

```mermaid
flowchart TB
    WriteData["Write Data"] --> RF
    WriteAddr["Write Addr [2:0]"] --> RF
    ReadAddr["Read Addr [2:0]"] --> RF
    WE["Write Enable"] --> RF
    
    subgraph RF[Register File]
        R0[R0]
        R1[R1]
        R2[R2]
        R3[R3]
        R4[R4]
        R5[R5]
        R6[R6]
        R7[R7]
    end
    
    RF --> ReadData["Read Data"]
```

## Exercise 2: Register File

In [ ]:
class RegisterFile:
    """Register file with 8 registers (R0-R7)."""

    def __init__(self, num_registers: int = 8):
        self.registers = [Register8() for _ in range(num_registers)]
        self.num_registers = num_registers

    def _addr_to_index(self, addr: List[int]) -> int:
        """Convert 3-bit address to integer index."""
        # YOUR CODE HERE
        # addr is [bit0, bit1, bit2], LSB first
        pass

    def read(self, addr: List[int]) -> List[int]:
        """Read from a register.

        Args:
            addr: 3-bit register address (LSB first)

        Returns:
            8-bit register value
        """
        # YOUR CODE HERE
        pass

    def write(self, addr: List[int], data: List[int], enable: int, clk: int) -> None:
        """Write to a register.

        Args:
            addr: 3-bit register address
            data: 8-bit data to write
            enable: Write enable signal
            clk: Clock signal
        """
        # YOUR CODE HERE
        pass

    def read_two(self, addr1: List[int], addr2: List[int]) -> tuple:
        """Read from two registers simultaneously (for ALU operations)."""
        return (self.read(addr1), self.read(addr2))


# Test
rf = RegisterFile()

# Write 10 to R3
addr_r3 = [1, 1, 0]  # 3 in binary (LSB first)
data = int_to_bits(10, 8)
rf.write(addr_r3, data, 1, 0)
rf.write(addr_r3, data, 1, 1)

# Write 20 to R5
addr_r5 = [1, 0, 1]  # 5 in binary
data = int_to_bits(20, 8)
rf.write(addr_r5, data, 1, 0)
rf.write(addr_r5, data, 1, 1)

# Read back
print(f"R3 = {bits_to_int(rf.read(addr_r3))}")
print(f"R5 = {bits_to_int(rf.read(addr_r5))}")

# Read two at once
a, b = rf.read_two(addr_r3, addr_r5)
print(f"R3 + R5 = {bits_to_int(a)} + {bits_to_int(b)} = {bits_to_int(a) + bits_to_int(b)}")

## Visualizing the Register File

In [ ]:
def dump_registers(rf: RegisterFile) -> None:
    """Print all register values."""
    print("Register File Contents:")
    print("-" * 25)
    for i in range(rf.num_registers):
        addr = [((i >> j) & 1) for j in range(3)]
        val = bits_to_int(rf.read(addr))
        print(f"  R{i}: {val:3d}  (0x{val:02X})  ({rf.read(addr)})")


# Initialize some values
rf = RegisterFile()
for i in range(8):
    addr = [((i >> j) & 1) for j in range(3)]
    data = int_to_bits(i * 10, 8)  # R0=0, R1=10, R2=20, ...
    rf.write(addr, data, 1, 0)
    rf.write(addr, data, 1, 1)

dump_registers(rf)

## Copy Your Implementation

Once your code works, copy the `Register8` and `RegisterFile` classes to `src/computer/registers.py`

## Validation

In [ ]:
from utils.checker import check

check("registers")

## Summary

We've built the CPU's fast local storage:

| Component | Description |
|-----------|-------------|
| Register8 | 8-bit storage from 8 D flip-flops |
| RegisterFile | 8 registers with address-based access |

### Key Concepts

1. **Registers** are groups of flip-flops that share a clock
2. **Enable** signals control which register gets written
3. **Register files** allow addressing multiple registers
4. CPUs use **two read ports** for ALU operands

### What's Next?

In the next notebook, we'll build **counters** - registers that automatically increment. The most important counter is the **Program Counter (PC)** that tracks which instruction to execute next!